# Estonia

In [4]:
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup

pd.options.mode.chained_assignment = None

## Fetch results

In [40]:
def get_links():
    base_url = 'http://ep2014.vvk.ee'
    r = requests.get(f'{base_url}/detailed.html')
    html = BeautifulSoup(r.text)
    links = html.select('.detailed-navigator-link > a')
    return [(x.text, f'{base_url}/{x["href"]}') for x in links][1:]

In [38]:
def get_turnout(county):
    df = pd.read_html(county[1], index_col=1)[0]
    df = df.drop('Reg nr', axis=1)
    turnouts = df.loc['Osavõtt %'].reset_index()
    turnouts['county'] = county[0]
    return turnouts

In [41]:
links = get_links()

In [44]:
municipalities = []

for link in links:
    try:
        municipalities.append(get_turnout(link))
    except KeyError:
        print(link)
        continue

('Tartu linn', 'http://ep2014.vvk.ee/detailed/0795/0795.html')
('Välisriigid', 'http://ep2014.vvk.ee/detailed/9999.html')


In [104]:
df = pd.concat(municipalities)
df.columns = ['municipality', 'turnout', 'county']

In [105]:
df['municipality'] = df['municipality'].apply(lambda x: ' '.join(x.split(' ')[:-1]) if x not in ['Viljandi vald', 'Põltsamaa vald', 'Jõgeva vald'] else x)

In [106]:
df.municipality = df.municipality.str.replace('Vändra vald', 'Vändra vald (alev)')

In [107]:
# Manually entered from web
df = df.append({
    'municipality': 'Tallinn',
    'turnout': 43.58,
    'county': 'Tallin'
}, ignore_index=True)

In [7]:
df.to_csv('../../data/raw/est.csv', index=False)

## Merge

In [18]:
df = df.rename(columns={'municipality': 'name'})

In [8]:
geo = gpd.read_file('../../data/shapefiles/est/est.shp')

In [19]:
geo = geo.merge(df, left_on='NAME_2', right_on='name', how='left')

## Export

In [23]:
geo = geo[['name', 'turnout', 'geometry']]

In [25]:
geo['country'] = 'est'

In [26]:
geo.to_file('../../data/processed/est')